# Tutorial

Kai Foerster, Amin Oueslati, Steve Kerr

## Introduction
Policy motivation: many institutions want to use something like ChatGPT but with their own domain knowledge <br>
Explain what a RAG chatbot is   <br>

### Next steps

	- Add HF token in .env

# Setup

* Install dependencies
* Configure an API key for Hugging Face

In [2]:
# install dependencies
!pipenv install langchain
!pipenv install sentence_transformers
!pipenv install chromadb
!pipenv install unstructured
!pipenv install chainlit
!pipenv install python-dotenv
!pipenv install bs4
!pipenv install tqdm

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchain...
Resolving langchain...
✔ Installation Succeeded
⠋ Installing langchain...
Installing dependencies from Pipfile.lock (1560fd)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 t

## Building a chatbot (no RAG)

In [3]:
import os
import chainlit as cl
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from dotenv import load_dotenv

2023-12-12 14:46:07 - Loaded .env file


In [5]:
load_dotenv()
HF_API_TOKEN = os.getenv('HF_API_TOKEN')

In [6]:
model_id = "tiiuae/falcon-7b-instruct"
conv_model = HuggingFaceHub(
    huggingfacehub_api_token=os.environ['HF_API_TOKEN'], 
    repo_id=model_id, 
    model_kwargs={"temperature":0.8,"max_length": 1000}
    )

/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [7]:
template="""You are a helpful assistant that answers questions of the user.
{human_message}
"""

prompt=PromptTemplate(template=template, input_variables=["human_message"])

In [8]:
conv_chain = LLMChain(llm=conv_model, prompt=prompt, verbose=True)

In [9]:
print(conv_chain.run("How much does a cappucino cost at Pret a Manger in Berlin Mitte?"))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user.
How much does a cappucino cost at Pret a Manger in Berlin Mitte?


> Finished chain.
The cost of a cappucino at Pret a Manger in Berlin Mitte varies by size, but generally ranges from 4.95 to 6.95 euros.


### Hallucinations

In [10]:
print(conv_chain.run("How much does a cappucino cost at Pret a Manger in Berlin Mitte?"))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user.
How much does a cappucino cost at Pret a Manger in Berlin Mitte?


> Finished chain.
The cost of a cappucino at Pret a Manger in Berlin Mitte varies by size, but generally ranges from 4.95 to 6.95 euros.


### Source knowledging (manual)

In [11]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

In [12]:
template_with_context="""You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:{source_knowledge}

{human_message}
"""

prompt2=PromptTemplate(template=template_with_context, input_variables=["human_message",  "source_knowledge"])

In [13]:
print(prompt2.format(human_message="What is a LLMChain?", source_knowledge=source_knowledge))

You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.
LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with

In [14]:
context_chain = LLMChain(llm=conv_model, prompt=prompt2, verbose=True)

In [15]:
print(context_chain.run({

  'source_knowledge': source_knowledge,

  'human_message': "What is Langchain?"

}))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.
LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of d

## RAG 
### Create database to store your corpus on

In [16]:
# load dependencies
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import shutil

In [17]:
# set params
DATA_PATH = "data/html"
CHROMA_PATH = "chroma_db"
EMBED_MODEL = "all-MiniLM-L6-v2" # Chroma defaults to "sentence-transformers/all-MiniLM-L6-v2"
# alternative: "BAAI/bge-small-en-v1.5"

# Load Documents

In [18]:
from bs4 import SoupStrainer
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import BSHTMLLoader

In [21]:
# define Beautiful Soup key word args
bs_kwargs = {
    "features": "html.parser", 
    "parse_only": SoupStrainer("p") # only include relevant text
}

# define Loader key word args
loader_kwargs = {
    "open_encoding": "utf-8",
    "bs_kwargs": bs_kwargs
}

# define Loader
loader = DirectoryLoader(
    path='data/html', 
    glob="*.html", 
    loader_cls=BSHTMLLoader,
    loader_kwargs=loader_kwargs,
    show_progress=True
    )

# load docs
documents = loader.load()
len(documents)

100%|██████████| 3487/3487 [00:04<00:00, 712.79it/s]


3487

In [22]:
# inspect first doc
documents[0]

Document(page_content='Warranties of data shall be developed and used in accordance with agency regulations.', metadata={'source': 'data/html/46.708.html', 'title': ''})

## Label Metadata

In [24]:
# add source label
import re

for doc in documents:
    doc_source = re.search("\d{1,2}[.]\d+(\-\d)*", doc.metadata["source"]).group() 
    doc.metadata["source"] = " ".join(["FAR", doc_source])

In [27]:
# define Beautiful Soup key word args
bs_kwargs = {
    "features": "html.parser", 
    "parse_only": SoupStrainer("title") # only include relevant text
}

# define Loader key word args
loader_kwargs = {
    "open_encoding": "utf-8",
    "bs_kwargs": bs_kwargs
}

loader = DirectoryLoader(
    path='data/html', 
    glob="*.html", 
    loader_cls=BSHTMLLoader,
    loader_kwargs=loader_kwargs,
    show_progress=True
    )

document_titles = loader.load()

# Convert the metadata for the specified label into a list
title_list = [doc.metadata["title"] for doc in document_titles]

# add title label
i = 0
for doc in documents:
    doc.metadata["title"] = title_list[i]
    i += 1

  0%|          | 0/3487 [00:00<?, ?it/s]

100%|██████████| 3487/3487 [00:03<00:00, 1006.60it/s]


In [ ]:
# # add FAR part label
# import re 

# for doc in docs:
#     doc_part = re.search('^(\d{1,2})', doc.metadata['source']).group()
#     doc.metadata["part"] = " ".join(["FAR Part", doc_part])
    
#     print(doc.metadata["part"])

In [30]:
# inspect metadata 
doc_metadata = [doc.metadata  for doc in documents]
doc_metadata[0:5]

[{'source': 'FAR 46.708', 'title': '46.708 Warranties of data.'},
 {'source': 'FAR 9.405', 'title': '9.405 Effect of listing.'},
 {'source': 'FAR 11.106',
  'title': '11.106 Purchase descriptions for service contracts.'},
 {'source': 'FAR 16.204', 'title': '16.204 Fixed-price incentive contracts.'},
 {'source': 'FAR 7.201', 'title': '7.201 [Reserved]'}]

# Embed Documents & Upload to Vector Database

In [ ]:
# define text embedding model
embedding_func = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)

# See https://huggingface.co/spaces/mteb/leaderboard

2023-12-11 15:32:58 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-12-11 15:32:59 - Use pytorch device: cpu


In [ ]:
# first, clear out current db
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

# initialize Chroma db and save locally
db = Chroma.from_documents(
    documents=documents, embedding=embedding_func, persist_directory=CHROMA_PATH
    )

db.persist()

# print message
print(f"Saved {len(documents)} chunks to {CHROMA_PATH}.")

2023-12-11 15:33:26 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.28it/s]

Saved 3487 chunks to chroma_db.


# Query Vector Database

In [ ]:
# query vector db
query = "What is the purpose of the Federal Acquisition Regulations?"
matching_docs = db.similarity_search_with_relevance_scores(
    query=query, 
    k=4, # number of docs to return
    #score_threshold=.5,
    #filter=[{"":""}]
    )

matching_docs

Batches: 100%|██████████| 1/1 [00:00<00:00, 34.49it/s]


[(Document(page_content='1.101 Purpose.\n\nThe Federal Acquisition Regulations System is established for the codification and publication of uniform policies and procedures for acquisition by all executive agencies. The Federal Acquisition Regulations System consists of the Federal Acquisition Regulation (FAR), which is the primary document, and agency acquisition regulations that implement or supplement the FAR. The FAR System does not include internal agency guidance of the type described in 1.301(a)(2).\n\nSubpart 1.1 - Purpose, Authority, Issuance', metadata={'source': 'data\\html\\1.101.html'}),
  0.754029959492634),
 (Document(page_content='1.000 Scope of part.\n\nThis part sets forth basic policies and general information about the Federal Acquisition Regulations System including purpose, authority, applicability, issuance, arrangement, numbering, dissemination, implementation, supplementation, maintenance, administration, and deviation. subparts\xa0 1.2,1.3, and 1.4 prescribe a

### Query data from your database based on your prompt

In [ ]:
### adapted version 
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def RAG(query_text):
    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=6)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    if len(context_text) > 1000:
        context_text = context_text[:1000]
        print("Warning: Context exceeded 1000 characters, trimming from the end.")

    prompt_template=PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context",  "question"])
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    chain = LLMChain(llm=conv_model, prompt=prompt_template, verbose=True)
    response_text = chain.run({"context": context_text, "question": query})
    
    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

### Parse the augumented prompt into the chatmodel

In [ ]:
RAG("What does the Federal Acquisition Regulations define?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 76.74it/s]




> Entering new LLMChain chain...
Prompt after formatting:

Answer the question based only on the following context:

1.101 Purpose.

The Federal Acquisition Regulations System is established for the codification and publication of uniform policies and procedures for acquisition by all executive agencies. The Federal Acquisition Regulations System consists of the Federal Acquisition Regulation (FAR), which is the primary document, and agency acquisition regulations that implement or supplement the FAR. The FAR System does not include internal agency guidance of the type described in 1.301(a)(2).

Subpart 1.1 - Purpose, Authority, Issuance

---

1.000 Scope of part.

This part sets forth basic policies and general information about the Federal Acquisition Regulations System including purpose, authority, applicability, issuance, arrangement, numbering, dissemination, implementation, supplementation, maintenance, administration, and deviation. subparts  1.2,1.3, and 1.4 prescribe adminis

### Human evaluation of RAG model
Do we wanna add some other evaluation methods here??

In [ ]:
prompt = HumanMessage(
    content="what safety measures were used in the development of llama 2?"
)

res = chat(messages + [prompt])
print(res.content)

In [ ]:
prompt = HumanMessage(
    content=augment_prompt(
        "what safety measures were used in the development of llama 2?"
    )
)

res = chat(messages + [prompt])
print(res.content)

## References

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb <br>
https://github.com/pixegami/langchain-rag-tutorial/tree/main <br>
https://www.youtube.com/watch?v=LhnCsygAvzY <br>
https://www.youtube.com/watch?v=tcqEUSNCn8I <br>
https://towardsdatascience.com/a-3-step-approach-to-evaluate-a-retrieval-augmented-generation-rag-5acf2aba86de <br>
https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain <br>
https://www.mlexpert.io/prompt-engineering/chatbot-with-local-llm-using-langchain <br>
https://www.youtube.com/watch?v=N7dGOUwufBM <br>
https://www.youtube.com/watch?v=ypzmPwLH_Q4&list=PLIUOU7oqGTLjAwPzyCu6m0wxLOlhJg8N5&index=2<br>
https://www.youtube.com/watch?v=qMIM7dECAkc <br>
https://www.youtube.com/watch?v=ukj_ITJKBwE&list=PLIUOU7oqGTLjAwPzyCu6m0wxLOlhJg8N5&index=6
